In [3]:
import pyspark

In [4]:
pyspark.__version__

'3.4.1'

In [6]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

## 1.  세션생성하기 

In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
           .config("spark.driver.host","127.0.0.1") 
           .config("spark.driver.bindAddress","127.0.0.1")
           .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/12 13:41:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/12 13:41:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/09/12 13:41:38 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
spark

## 2. 파이스파크 데이터프레임 생성하기 

- 파이스파크는 트랜스포머와 액션 두 가지의 함수나 메서드를 제공
- 실제 트랜스포머는 지연처리이고 액션이 실행되어야 전체가 실행된다.

## 2-1 스파크에서 제공하는 Row 클래스 사용하기 

In [7]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])

In [8]:
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [9]:
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



### 액션 실행
- show, count, collect는 주요 액션처리
- 또는 파일에 저장등이 액션이다 

In [10]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



## 2-2 일반적인 파이선 리스트로 파이스파크 데이터프레임 생성 가능 

- 스키마 정보를 문자열로 지정해서 처리
- 'a long, b double, c string, d date, e timestamp' : 변수명과 타입을 쌍으로 처리 

### 출력된 결과를 판다스 처럼 보기 위한 세팅 

In [14]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

### 데이터를 리스트 및 스키마 정보를 별도 부여해서 데이터프레임 생성하기 

In [15]:
df1 = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')


In [16]:
df1

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [17]:
df1.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+

